In [1]:
!pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
  

## Configs

In [3]:
LABEL2IDX = {

    'B_DAT': 0,
    'B_EVE': 1,
    'B_FAC': 2,
    'B_LOC': 3,
    'B_MON': 4,
    'B_ORG': 5,
    'B_PCT': 6,
    'B_PER': 7,
    'B_PRO': 8,
    'B_TIM': 9,
    'I_DAT': 10,
    'I_EVE': 11,
    'I_FAC': 12,
    'I_LOC': 13,
    'I_MON': 14,
    'I_ORG': 15,
    'I_PCT': 16,
    'I_PER': 17,
    'I_PRO': 18,
    'I_TIM': 19,
    'O': 20,
}

IDX2LABEL = {i: k for k, i in LABEL2IDX.items()}

MAX_LEN = 128
TRAIN_BATCH_SIZE = 8
EVAL_BATCH_SIZE = 8
EPOCHS = 10
NUM_CLASS = 21
LEARNING_RATE = 2e-5

In [4]:
from datasets import concatenate_datasets, load_dataset
from transformers import AutoTokenizer
import ast

model_name = 'sbunlp/fabert'
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/18.3k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/552k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [5]:
def convert_string_to_list(example):
    def convert_column(column):
        # Check if the column is a string and process accordingly
        if isinstance(column, str):
            column = column.strip("[]")
            items = column.split("' '")
            items = [item.strip().strip("'") for item in items if item.strip()]
            return items
        return column

    def convert_numeric_column(column):
        # Check if the column is a string and process accordingly
        if isinstance(column, str):
            column = column.strip("[]")
            items = column.split(" ")
            items = [int(item.strip()) for item in items if item.strip().isdigit()]
            return items
        return column

    # Process all relevant columns
    example['tokens'] = convert_column(example['tokens'])
    example['ner_tags'] = convert_numeric_column(example['ner_tags'])
    example['ner_tags_names'] = convert_column(example['ner_tags_names'])

    return example

In [6]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"],truncation=True,max_length=512,
 is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [7]:
dataset = load_dataset('AliFartout/PEYMA-ARMAN-Mixed')
dataset = dataset.remove_columns('Unnamed: 0')
dataset = concatenate_datasets([dataset['train'], dataset['validation'], dataset['test']])
dataset

Generating train split:   0%|          | 0/26417 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3302 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3303 [00:00<?, ? examples/s]

Dataset({
    features: ['tokens', 'ner_tags', 'ner_tags_names'],
    num_rows: 33022
})

In [8]:
dataset = dataset.map(convert_string_to_list)

Map:   0%|          | 0/33022 [00:00<?, ? examples/s]

In [9]:
print(type(dataset[50]['tokens']))
print(type(dataset[2]['ner_tags']))
print((dataset[2]['ner_tags_names']))

<class 'list'>
<class 'list'>
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', "O'\n 'B_EVE", 'I_EVE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', "O'\n 'O", 'O', 'O', 'O', 'O', 'O', 'O']


In [10]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True, remove_columns=dataset.column_names,)

Map:   0%|          | 0/33022 [00:00<?, ? examples/s]

In [11]:
tokenized_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 33022
})

In [12]:
# Printing the values and lengths of each column in the first item of tokenized_dataset
print(f"Input IDs: {tokenized_dataset[0]['input_ids']}, length: {len(tokenized_dataset[0]['input_ids'])}")
print(f"Token Type IDs: {tokenized_dataset[0]['token_type_ids']}, length: {len(tokenized_dataset[0]['token_type_ids'])}")
print(f"Attention Mask: {tokenized_dataset[0]['attention_mask']}, length: {len(tokenized_dataset[0]['attention_mask'])}")
print(f"Labels: {tokenized_dataset[0]['labels']}, length: {len(tokenized_dataset[0]['labels'])}")



Input IDs: [101, 44516, 34834, 6840, 20039, 2317, 2472, 6034, 2402, 23427, 2713, 117, 102], length: 13
Token Type IDs: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], length: 13
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], length: 13
Labels: [-100, 7, -100, 20, 3, 20, 20, 20, 20, 3, 20, 20, -100], length: 13


In [13]:
 print(tokenizer.decode(tokenized_dataset[0]['input_ids'][0]))
 print(tokenizer.decode(tokenized_dataset[0]['input_ids'][1]))
 print(tokenizer.decode(tokenized_dataset[0]['input_ids'][-2]))
 print(tokenizer.decode(tokenized_dataset[0]['input_ids'][-1]))

[CLS]
یوک
.
[SEP]


In [14]:
print(tokenizer.convert_ids_to_tokens(tokenized_dataset[0]['input_ids']))


['[CLS]', 'یوک', '##وویچ', 'متولد', 'دانمارک', 'است', 'اما', 'والدین', 'او', 'صرب', 'هستند', '.', '[SEP]']


In [15]:

train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_eval_split = train_test_split['test'].train_test_split(test_size=0.5)

train_dataset = train_test_split['train']
eval_dataset = train_eval_split['train']
test_dataset = train_eval_split['test']
# Print out info of each dataset
print(f"Training set: {train_dataset} samples")
print(f"Validation set: {eval_dataset} samples")
print(f"Test set: {test_dataset} samples")


Training set: Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 26417
}) samples
Validation set: Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 3302
}) samples
Test set: Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 3303
}) samples


In [16]:
from transformers import DataCollatorForTokenClassification
import torch
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [17]:
train_dataset.set_format("torch")
eval_dataset.set_format("torch")
test_dataset.set_format("torch")

In [32]:
train_sample = train_dataset[0]
print(type(train_sample['input_ids']))

<class 'torch.Tensor'>


In [31]:
# check the paddings with -100s
batch = data_collator([train_dataset[i] for i in range(2)])
batch["labels"]

tensor([[-100,   20,    5,   15,   15,   15,   20,   20,   20,   20,   20,   20,
         -100, -100, -100,   20,   20, -100, -100, -100, -100, -100,   20,   20,
           20, -100, -100, -100, -100, -100, -100,   20,   20, -100, -100, -100,
           20,   20, -100, -100, -100, -100, -100,   20,   20, -100,   20, -100,
           20, -100, -100, -100, -100, -100, -100, -100, -100, -100,   20,   20,
           20,   20,   20,   20,   20,   20,   20,   20, -100, -100, -100,   20,
         -100],
        [-100,   20,    5,   15,   15,   15,   20,   20,   20,   20,   20, -100,
         -100, -100,   20,   20,   20,   20,   20,   20,   20,   20,   20,   20,
         -100, -100, -100,   20,   20,   20,   20,    3,   20,   20,   20,   20,
         -100, -100, -100,    5,   15,   15,   15,   20,   20,   20, -100, -100,
         -100, -100, -100,   20, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -10

In [30]:
for i in range(2):
    print(train_dataset[i]["labels"])

tensor([-100,   20,    5,   15,   15,   15,   20,   20,   20,   20,   20,   20,
        -100, -100, -100,   20,   20, -100, -100, -100, -100, -100,   20,   20,
          20, -100, -100, -100, -100, -100, -100,   20,   20, -100, -100, -100,
          20,   20, -100, -100, -100, -100, -100,   20,   20, -100,   20, -100,
          20, -100, -100, -100, -100, -100, -100, -100, -100, -100,   20,   20,
          20,   20,   20,   20,   20,   20,   20,   20, -100, -100, -100,   20,
        -100])
tensor([-100,   20,    5,   15,   15,   15,   20,   20,   20,   20,   20, -100,
        -100, -100,   20,   20,   20,   20,   20,   20,   20,   20,   20,   20,
        -100, -100, -100,   20,   20,   20,   20,    3,   20,   20,   20,   20,
        -100, -100, -100,    5,   15,   15,   15,   20,   20,   20, -100, -100,
        -100, -100, -100,   20, -100])


In [ ]:
IDX2LABEL

{0: 'O',
 1: 'B-DAT',
 2: 'B-PER',
 3: 'B-ORG',
 4: 'B-LOC',
 5: 'B-EVE',
 6: 'I-DAT',
 7: 'I-PER',
 8: 'I-ORG',
 9: 'I-LOC',
 10: 'I-EVE'}

In [ ]:
LABEL2IDX

{'O': 0,
 'B-DAT': 1,
 'B-PER': 2,
 'B-ORG': 3,
 'B-LOC': 4,
 'B-EVE': 5,
 'I-DAT': 6,
 'I-PER': 7,
 'I-ORG': 8,
 'I-LOC': 9,
 'I-EVE': 10}

In [20]:
!pip install seqeval evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.4 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=a0dfd1c1da41fe835245bb77261c3ea38273ee46b5479c4e3087fce2e2864dad
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [18]:
label_list = [k for k,v in LABEL2IDX.items()]
label_list

['B_DAT',
 'B_EVE',
 'B_FAC',
 'B_LOC',
 'B_MON',
 'B_ORG',
 'B_PCT',
 'B_PER',
 'B_PRO',
 'B_TIM',
 'I_DAT',
 'I_EVE',
 'I_FAC',
 'I_LOC',
 'I_MON',
 'I_ORG',
 'I_PCT',
 'I_PER',
 'I_PRO',
 'I_TIM',
 'O']

In [21]:
import numpy as np
from seqeval.metrics import classification_report
import evaluate

accuracy = evaluate.load('accuracy')
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    #############
    cr = classification_report(true_labels, true_predictions,digits=4)
    fname = "report.txt"
    file = open(fname,'w')
    file.write(cr)
    file.close()
    #############
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [28]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, AutoConfig
config = AutoConfig.from_pretrained(model_name)
config.attention_probs_dropout_prob = 0.2  #  default (
config.hidden_dropout_prob = 0.2  # default (0.1)
config.num_labels = NUM_CLASS
config.id2label = IDX2LABEL
config.label2id = LABEL2IDX
model = AutoModelForTokenClassification.from_pretrained(model_name,  config=config)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at sbunlp/fabert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
samples = train_dataset[:8]
batch = data_collator([train_dataset[i] for i in range(i)])
{k: v.shape for k, v in batch.items()}


{'input_ids': torch.Size([1, 73]),
 'token_type_ids': torch.Size([1, 73]),
 'attention_mask': torch.Size([1, 73]),
 'labels': torch.Size([1, 73])}

In [34]:
 model.config.max_position_embeddings

512

In [35]:
print(train_dataset)
print(eval_dataset)

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 26417
})
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 3302
})


In [37]:
from huggingface_hub import notebook_login

notebook_login()

In [38]:
from transformers import EarlyStoppingCallback
training_args = TrainingArguments(
    "faBERT-peymaarman-finetuned",
    learning_rate= 2e-5,
    per_device_train_batch_size= TRAIN_BATCH_SIZE,
    per_device_eval_batch_size= EVAL_BATCH_SIZE,
    num_train_epochs= 10,
    weight_decay= 0.025,#*********************
    warmup_ratio = 0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    lr_scheduler_type="linear",
    greater_is_better=True,
    push_to_hub=True,
    )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    )
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.250900,0.225813,0.515160,0.611902,0.559379,0.926808
2,0.115300,0.114684,0.722997,0.790945,0.755446,0.964387
3,0.075000,0.086030,0.799005,0.854227,0.825694,0.974750
4,0.049700,0.080871,0.819400,0.870691,0.844267,0.977628
5,0.039900,0.072841,0.860183,0.901046,0.880141,0.982815
6,0.029600,0.070661,0.861702,0.916824,0.888409,0.982908
7,0.022900,0.066233,0.883882,0.921626,0.902359,0.985082
8,0.016700,0.061294,0.901108,0.934488,0.917495,0.987463
9,0.015100,0.062787,0.903274,0.936889,0.919774,0.987411
10,0.011900,0.064103,0.903764,0.938947,0.921019,0.987442


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_PRO seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: I_ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_EVE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.p

TrainOutput(global_step=33030, training_loss=0.10599461312623158, metrics={'train_runtime': 3711.7942, 'train_samples_per_second': 71.17, 'train_steps_per_second': 8.899, 'total_flos': 1.3563741459188304e+16, 'train_loss': 0.10599461312623158, 'epoch': 10.0})

In [40]:
log_history = trainer.state.log_history
log_history

[{'loss': 1.958,
  'grad_norm': 5.871665000915527,
  'learning_rate': 3.0275507114744173e-06,
  'epoch': 0.15137753557372086,
  'step': 500},
 {'loss': 0.6269,
  'grad_norm': 1.4234650135040283,
  'learning_rate': 6.055101422948835e-06,
  'epoch': 0.3027550711474417,
  'step': 1000},
 {'loss': 0.5437,
  'grad_norm': 2.828749418258667,
  'learning_rate': 9.082652134423252e-06,
  'epoch': 0.45413260672116257,
  'step': 1500},
 {'loss': 0.3919,
  'grad_norm': 5.825778484344482,
  'learning_rate': 1.211020284589767e-05,
  'epoch': 0.6055101422948834,
  'step': 2000},
 {'loss': 0.31,
  'grad_norm': 5.009946346282959,
  'learning_rate': 1.5137753557372088e-05,
  'epoch': 0.7568876778686043,
  'step': 2500},
 {'loss': 0.2509,
  'grad_norm': 4.9603471755981445,
  'learning_rate': 1.8165304268846504e-05,
  'epoch': 0.9082652134423251,
  'step': 3000},
 {'eval_loss': 0.22581252455711365,
  'eval_precision': 0.5151602656656079,
  'eval_recall': 0.6119019036185903,
  'eval_f1': 0.5593791643803402,

In [39]:
model.config.num_labels

21

In [41]:
validation_results = trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_PRO seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: I_ORG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_EVE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.p

In [42]:
validation_results

{'eval_loss': 0.06410308182239532,
 'eval_precision': 0.903763618355893,
 'eval_recall': 0.9389470073743783,
 'eval_f1': 0.9210194297249558,
 'eval_accuracy': 0.9874424142036338,
 'eval_runtime': 13.186,
 'eval_samples_per_second': 250.417,
 'eval_steps_per_second': 31.321,
 'epoch': 10.0}

In [43]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
        )
test_results = trainer.evaluate()

In [44]:
test_results

{'eval_loss': 0.06467791646718979,
 'eval_precision': 0.9017072766451185,
 'eval_recall': 0.934386808656819,
 'eval_f1': 0.9177562210037958,
 'eval_accuracy': 0.9878789776998246,
 'eval_runtime': 13.3901,
 'eval_samples_per_second': 246.674,
 'eval_steps_per_second': 30.844}

In [45]:
with open('report.txt') as f:
    for line in f.readlines():
        print(line)

              precision    recall  f1-score   support



        _DAT     0.7415    0.7716    0.7562       197

        _EVE     0.7914    0.9867    0.8783       150

        _FAC     0.8092    0.9248    0.8632       133

        _LOC     0.9193    0.9199    0.9196      1635

        _MON     0.6905    0.9062    0.7838        32

        _ORG     0.9089    0.9474    0.9278      1959

        _PCT     0.7838    0.9355    0.8529        31

        _PER     0.9280    0.9465    0.9371      1457

        _PRO     0.9249    0.9949    0.9586       198

        _TIM     0.6970    0.7667    0.7302        30



   micro avg     0.9017    0.9344    0.9178      5822

   macro avg     0.8194    0.9100    0.8608      5822

weighted avg     0.9032    0.9344    0.9181      5822

